In [1]:
%load_ext autoreload
%autoreload 2

### Lead Sheet to Multi-Track Arrangement

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']= '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import pretty_midi as pyd
from arrangement_utils import *
import warnings
warnings.filterwarnings("ignore")

"""Download checkpoints and data at https://drive.google.com/file/d/1ZyswS0p_t2Ij5vyaFkM5IbVgphf78oTB/view?usp=sharing and decompress at the directory"""
DATA_FILE_ROOT = './data_file_dir/' #configure this dir to where you've saved and decompressed the zip
DEVICE = 'cuda:0'
piano_arranger, orchestrator, piano_texture, band_prompt = load_premise(DATA_FILE_ROOT, DEVICE)

Loading lead sheet to piano arrangement module (piano arranger). This may take 1 or 2 minutes ...


100%|██████████| 15/15 [00:00<00:00, 18.48it/s]


Loading piano to multi-track arrangement module (orchestrator). This may take 1 or 2 minutes ...
loading Slakh2100 Dataset ...


100%|██████████| 331/331 [00:02<00:00, 129.19it/s]


Initializing prior model
Finished.


### Initialize input
We provide a few sample lead sheets for a quick inference. You can run the code blocks after downloading the pre-trained checkpoints.

If you wish to test our model on your own lead sheet file, please initialize a sub-folder with its `SONG_NAME` in the `./demo` folder and put the file in, and name the file "lead sheet.mid". 

Please also specify `SEGMENTATION` (phrase structure) and `NOTE_SHIFT` (the number of pick-up beats if any).

In [ ]:
"""Set input lead sheet"""
SONG_NAME, SEGMENTATION, NOTE_SHIFT, TEMPO = '01 Can You Feel The Love Tonight', 'i4A8B8B8x4A8B8B8O4', 0, 65
#SONG_NAME, SEGMENTATION, NOTE_SHIFT, TEMPO = '02 All I Want For Christmas Is You', 'A16B11C2A16B11C2D16E6C2O8', 0, 140
#SONG_NAME, SEGMENTATION, NOTE_SHIFT, TEMPO = '03 Boogie woogie bugle boy', 'A12A12B12', 4, 120
#SONG_NAME, SEGMENTATION, NOTE_SHIFT, TEMPO = '04 Castles in the Air', 'A8A8B8B8', 1 , 100  #1 beat in the pick-up measure
#SONG_NAME, SEGMENTATION, NOTE_SHIFT, TEMPO = '05 Jingle Bells', 'A8B8A8', 0, 100
#SONG_NAME, SEGMENTATION, NOTE_SHIFT, TEMPO = '06 Sally Garden', 'A4A4B4A4', 0, 75

lead_sheet = read_lead_sheet('./demo', SONG_NAME, SEGMENTATION, NOTE_SHIFT)

### Piano Accompaniment Arrangement

Stage 1: piano accompaniment arrangement from lead sheet.

In [ ]:
"""Set texture prefilter for piano arranger"""
RHTHM_DENSITY = 2   #np.random.randint(2, 5)
VOICE_NUMBER = 3    #np.random.randint(2, 5)
PREFILTER = (RHTHM_DENSITY, VOICE_NUMBER)

midi_piano, acc_piano = piano_arrangement(*lead_sheet, *piano_texture, piano_arranger, PREFILTER, TEMPO)
midi_path = f'./demo/{SONG_NAME}/arrangement_piano.mid'
midi_piano.write(midi_path)

### Orchestration

Stage 2: multi-track orchestration from piano

We need to decide using what instruments for orchestration. Here, we sample a piece from Slakh2100 dataset and borrow its instruments.

You can choose which instruments you wish to have or not by passing thier program number to the `MUST_HAVE` or `MUSTNOT_HAVE` list. We support 34 instrument classes as listed [here](orchestrator/autoencoder_dataset.py).

In [ ]:
MUST_HAVE = [0, 24] # 0: Acoustic Piano;24: Acoustic Guitar;
MUSTNOT_HAVE = [64] # 64: Soprano/Alto Sax
func_prompt = prompt_sampling(acc_piano, *band_prompt, MUST_HAVE, MUSTNOT_HAVE, DEVICE)

"""Set if use a 2-bar orchestral prompt for full-band arrangement (default True)""" 
USE_PROMPT = True

Now we orchestrate the piano arrangement from Stage 1 to a multi-track arrangement.

In [ ]:
if USE_PROMPT:
    instruments, time_promt = func_prompt
else:
    instruments, _ = func_prompt
    time_promt = None
midi_collection = orchestration(acc_piano, None, instruments, time_promt, orchestrator, DEVICE, blur=.25, p=.05, t=6, tempo=TEMPO, num_sample=4)

mel_track = pyd.Instrument(program=82, is_drum=False, name='melody')
mel_track.notes = midi_piano.instruments[0].notes
for idx, piece in enumerate(midi_collection):
    piece.instruments = [mel_track] + piece.instruments
    midi_path = f'./demo/{SONG_NAME}/arrangement_band-{str(idx).zfill(2)}.mid'
    piece.write(midi_path)